In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import collections

In [6]:
preverbsDF = pd.read_csv('../data/WiyotPreversText.tsv',sep='\t')
preverbsDF.head()

,Wiyot,English
0,bał/wał/wáł,relating.to
1,be/bé/béh,REP
2,bi/mi/mí,soon
3,bime/mime/bige/bihge/mige,some
4,bo/bó/wo/wó/ba/wa/boh,go.to


In [7]:
preverbs = list(set(preverbsDF['English']))
len(preverbs)

77

In [8]:
kroeber = pd.read_excel(f'../data/Kroeber texts.xlsx',None)
reichard = pd.read_excel(f'../data/Reichard texts.xlsx',None)
teeter = pd.read_excel(f'../data/Teeter texts.xlsx',None)

/Users/leonjdaeim/opt/miniconda3/envs/EDA/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [9]:
def processLine(x):
    try:
        x = '\t'.join(x.dropna()).split('\t')
        return x
    except:
        return None
    

## Remove empty lines
import re
matchEndPeriod = re.compile(r'\.$')
getlowercase = lambda x: x if x.isupper() else x.lower()    
_isna = lambda x: x.isna()

def getWordPairs(phrase):
    wordpairlist = []
    for i in range(0,len(phrase)-1):
        wordpairlist.append('|'.join([phrase[i],phrase[i+1]]))
    return wordpairlist

def countFrequencyWordPairs(listofwordpairs):
    try:
        return collections.Counter(listofwordpairs[:])
    except:
        return None

In [36]:
def processStory(author,story):
    data = author[story]
    data = data[~data.apply(_isna,0).all(1)]
    #####
    wiyotList = []
    originalEnglishList = []
    processedEnglishList = []
    preverbsEnglishList = []
    pairsEnglishList = []
    ######
    for i in range(0,len(data),2):   
        ##########################################
        wiyotPhrase = data.iloc[i]
        englishPhrase = data.iloc[i+1]
        wiyotPhrase = processLine(wiyotPhrase)    
        ### Do some extra processing to the english phrase
        englishPhrase = processLine(englishPhrase)
        #englishPhrase = [getlowercase(x) for x in englishPhrahaha se]
        ### Remove EMP = empty, remove periods at the end of gloss
        englishPhraseRedux = [getlowercase(matchEndPeriod.sub('',x)) for x in englishPhrase if x !='EMP'] #Remove EMP for empty 
        englishPreverbs = [_ for _ in englishPhraseRedux if _ in preverbs] 

        ### Get word pairs
        wps = getWordPairs(englishPhraseRedux)

        wiyotList.append(wiyotPhrase)
        originalEnglishList.append(englishPhrase)
        processedEnglishList.append(englishPhraseRedux)
        preverbsEnglishList.append(englishPreverbs)
        pairsEnglishList.append(wps)
        
    outDict = {"wiyot":wiyotList,
               "english_original":originalEnglishList,
              "english_clean":processedEnglishList,
              "english_preverbs":preverbsEnglishList,
              'wordPairs':pairsEnglishList}
    return outDict

def getPreverbs(dataset):
    try:
        #dPrev = {k:v['english_preverbs'] for k,v in dataset.items()}
        #return pd.DataFrame.from_dict(dPrev,orient='index')
        author_preverbs = [dataset[k]['english_preverbs'] for k in dataset.keys()]
        author_preverbs = sum(sum(author_preverbs,[]),[])
        return author_preverbs
    except:
        return []

In [13]:
kroeber_processed = {story:processStory(kroeber,story) for story in kroeber.keys()}
reichard_processed = {story:processStory(reichard,story) for story in reichard.keys()}
teeter_processed = {story:processStory(teeter,story) for story in teeter.keys()}

### Get preverb frequency

In [38]:
reichard_preverbs = getPreverbs(reichard_processed)
teeter_preverbs = getPreverbs(teeter_processed)
kroeber_preverbs = getPreverbs(kroeber_processed)

In [41]:
allpreverbs = kroeber_preverbs + reichard_preverbs + teeter_preverbs

In [42]:
import collections

In [61]:
allpreverbs_frequencies = collections.Counter(allpreverbs)
allpreverbs_frequencies = pd.DataFrame.from_dict(allpreverbs_frequencies,orient='index',columns=['frequency'])
allpreverbs_frequencies = allpreverbs_frequencies.sort_values('frequency',ascending=False).reset_index().rename(columns={'index':'preverb'})
allpreverbs_frequencies.head()

,preverb,frequency
0,PST,1335
1,PROG,1317
2,INCH,834
3,ITER,784
4,EMPH,508


In [111]:
allpreverbs_frequencies.to_csv('../results/allpreverbs_frequencies.txt',sep='\t',index=None)
#allpreverbs_frequencies.to_clipboard(index=None)

In [68]:
[dataset[k]['english_preverbs'] for k in dataset.keys()] 

{'wiyot': [['Kacid', 'wule', 'póyakhd', 'ił', 'šudi', 'lákhow'],
  ['wule', 'póyakhd', 'ił', 'hi', 'wú', 'lag', 'ił.', 'Šudi', 'lákhow'],
  ['hi', 'lès', 'ił.', 'Cákhi', 'tu', 'lùl', 'úw', 'ił'],
  ['díkh',
   'ad.',
   'Wuphedakáčhkš',
   'ił',
   'phlét',
   'w',
   'akw.',
   'Khedawakátkhadi'],
  ['wuphedakáckš',
   'ił.',
   'Hi',
   'khu',
   'lów',
   'ił.',
   'Tiwe',
   'dú',
   'lakh',
   'am',
   'e'],
  ['cèk?',
   'Khacid',
   'iél',
   'ił',
   'Kúckhed',
   'ehl',
   'tákw',
   'ta',
   'dú',
   'takw.'],
  ['Kacid', 'til', 'wíw', 'al.', 'Ku', 'khún', 'an'],
  ['ku', 'lúw', 'ił', 'cèk.', 'Kacidi', 'kótam', 'íł.', 'Tíle'],
  ['táwim', 'ił', 'ku', 'dáčhečhk.', 'Kwóčh', 'el'],
  ['yá', 'ki', 'them', 'óth', 'ił', 'Yil', 'il', 'phlet', 'w', 'ak'],
  ['tewu', 'them', 'ół', 'el.', 'Hi', 'lák', 'ił', 'til.'],
  ['Hi', 'dókhš', 'ém', 'ił.', 'Hi', 'wó', 'khu', 'dałé', 'khan', 'em', 'ił.'],
  ['Kacidi', 'hi', 'khu', 'lótw', 'ui.', 'Til', 'ta', 'néd', 'ił.'],
  ['Mes', 'akw', 'hi', 

In [95]:
def getPreverbsPairs(dataset):
    try:
        author_wordpairs = [[getWordPairs(_) for _ in dataset[k]['english_preverbs'] if len(_) >= 2] for k in dataset.keys()]
        author_wordpairs = sum(author_wordpairs,[])
        return author_wordpairs
    except:
        return []

In [99]:
kroeber_preverbPairs = getPreverbsPairs(kroeber_processed)
teeter_preverbPairs = getPreverbsPairs(teeter_processed)
reichard_preverbPairs = getPreverbsPairs(reichard_processed)

In [102]:
allPreverbPairs = sum(kroeber_preverbPairs + teeter_preverbPairs + reichard_preverbPairs,[])

In [109]:
allpreverbs_wordPair_frequencies = collections.Counter(allPreverbPairs)
allpreverbs_wordPair_frequencies = pd.DataFrame.from_dict(allpreverbs_wordPair_frequencies,orient='index',columns=['frequency'])
allpreverbs_wordPair_frequencies = allpreverbs_wordPair_frequencies.sort_values('frequency',ascending=False).reset_index().rename(columns={'index':'preverb_pair'})
allpreverbs_wordPair_frequencies.insert(0,'word_1',allpreverbs_wordPair_frequencies['preverb_pair'].map(lambda x: x.split('|')[0]))
allpreverbs_wordPair_frequencies.insert(1,'word_2',allpreverbs_wordPair_frequencies['preverb_pair'].map(lambda x: x.split('|')[1]))
allpreverbs_wordPair_frequencies.head()
allpreverbs_wordPair_frequencies.to_csv('../results/allpreverbs_pairs_frequencies.txt',sep='\t',index=None)

In [110]:
allpreverbs_wordPair_frequencies.to_clipboard(index=None)

In [107]:
len(allpreverbs_wordPair_frequencies)

651

In [108]:
allpreverbs_wordPair_frequencies.tail()

,word_1,word_2,preverb_pair,frequency
646,REP,EMPH,REP|EMPH,1
647,REP,INCH,REP|INCH,1
648,REP,ITER,REP|ITER,1
649,EMPH,some,EMPH|some,1
650,ITER,thoroughly,ITER|thoroughly,1
